# Using biomed.sm.mv-te-84m Models Via SmallMoleculeMultiViewModel API

In [1]:
# Necessary Imports
from bmfm_sm.api.smmv_api import SmallMoleculeMultiViewModel, PredictionIterator
from bmfm_sm.core.data_modules.namespace import LateFusionStrategy
from bmfm_sm.api.dataset_registry import DatasetRegistry

from dataclasses import asdict
from itertools import islice
import pandas as pd
import os

/Users/psuryan/Partha/work/code/envs/bmfm-sm-v5/lib/python3.11/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/psuryan/Partha/work/code/envs/bmfm-sm-v5/lib/python3.11/site-packages/libpyg.so, 0x0006): tried: '/Users/psuryan/Partha/work/code/envs/bmfm-sm-v5/lib/python3.11/site-packages/libpyg.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/psuryan/Partha/work/code/envs/bmfm-sm-v5/lib/python3.11/site-packages/libpyg.so' (no such file), '/Users/psuryan/Partha/work/code/envs/bmfm-sm-v5/lib/python3.11/site-packages/libpyg.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
2024-10-26 10:27:52,484 - rdkit - INFO - MacBookPro:8512582208:0:0 - Enabling RDKit 2024.03.5 jupyter extensions


## Explore the data

Following are the datasets available for evaluation, finetuning and inference.

In [2]:
pd.DataFrame(DatasetRegistry.get_instance().get_collection('MoleculeNet'))

,dataset_name,num_tasks,task_type,description,preferred_metric,path,example,collection,num_classes
0,BACE,1,TaskType.CLASSIFICATION,MoleculeNet: Inhibition of human beta secretase 1,Metrics.ROCAUC,datasets/raw_data/MoleculeNet/bace.csv,"CC(C)CC1=CC=C(C=C1)C(C)C(=O)O,0",DatasetCollection.MOLECULENET,2.0
1,BBBP,1,TaskType.CLASSIFICATION,MoleculeNet: Blood brain barrier penetration,Metrics.ROCAUC,datasets/raw_data/MoleculeNet/bbbp.csv,"CC(C)CC1=CC=C(C=C1)C(C)C(=O)O,0",DatasetCollection.MOLECULENET,2.0
2,CLINTOX,2,TaskType.CLASSIFICATION,MoleculeNet: Toxicity data of FDA-approved dru...,Metrics.ROCAUC,datasets/raw_data/MoleculeNet/clintox.csv,"[N+](=O)([O-])[O-],1 0",DatasetCollection.MOLECULENET,2.0
3,ESOL,1,TaskType.REGRESSION,MoleculeNet: Water solubility data for organics,Metrics.RMSE,datasets/raw_data/MoleculeNet/esol.csv,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,DatasetCollection.MOLECULENET,NaN
4,FREESOLV,1,TaskType.REGRESSION,MoleculeNet: Hydration free energy,Metrics.RMSE,datasets/raw_data/MoleculeNet/freesolv.csv,"CN(C)C(=O)c1ccc(cc1)OC,-11.01",DatasetCollection.MOLECULENET,NaN
5,HIV,1,TaskType.CLASSIFICATION,MoleculeNet: Inhibition of HIV viral replication,Metrics.ROCAUC,datasets/raw_data/MoleculeNet/hiv.csv,"CC(C)CC1=CC=C(C=C1)C(C)C(=O)O,0",DatasetCollection.MOLECULENET,2.0
6,LIPOPHILICITY,1,TaskType.REGRESSION,MoleculeNet: Octonol/water distribution coefff...,Metrics.RMSE,datasets/raw_data/MoleculeNet/lipophilicity.csv,"Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54",DatasetCollection.MOLECULENET,NaN
7,MUV,17,TaskType.CLASSIFICATION,MoleculeNet: PubChem derived target-activity t...,Metrics.ROCAUC,datasets/raw_data/MoleculeNet/muv.csv,Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1...,DatasetCollection.MOLECULENET,2.0
8,PCBA,128,TaskType.CLASSIFICATION,MoleculeNet: PubChem derived target-activity (...,Metrics.ROCAUC,datasets/raw_data/MoleculeNet/pcba.csv,"CC(=O)N1CCC2(CC1)NC(=O)N(c1ccccc1)N2,0 0 -1 0 ...",DatasetCollection.MOLECULENET,2.0
9,QM7,1,TaskType.REGRESSION,MoleculeNet: Electronic properties derived fro...,Metrics.MAE,datasets/raw_data/MoleculeNet/qm7.csv,"C([H])([H])([H])[H],-417.96",DatasetCollection.MOLECULENET,NaN


Get more information about a particular dataset

In [3]:
dataset = DatasetRegistry.get_instance().get_dataset_info('TOX21')
pd.DataFrame([asdict(dataset)])

,dataset_name,num_tasks,task_type,description,preferred_metric,path,example,collection,num_classes
0,TOX21,12,TaskType.CLASSIFICATION,MoleculeNet: Toxicity against set of targets,Metrics.ROCAUC,datasets/raw_data/MoleculeNet/tox21.csv,"CCOc1ccc2nc(S(N)(=O)=O)sc2c1,0 0 1 -1 -1 0 0 1...",DatasetCollection.MOLECULENET,2


## Using Models from HuggingFace

We have made pretrained and finetuned models available in [HuggingFace](https://huggingface.co/ibm/biomed.sm.mv-te-84m)

### Loading Pretrained Model from HuggingFace

Load a pretrained model from Huggingface by setting `model_path` to a Huggingface repo, and setting the `huggingface` argument to `True`

In [4]:
model = SmallMoleculeMultiViewModel.from_pretrained(LateFusionStrategy.ATTENTIONAL,
                                                    model_path='ibm/biomed.sm.mv-te-84m',
                                                    huggingface=True)

2024-10-26 10:27:53,309 - root - INFO - MacBookPro:8512582208:0:0 - Loading checkpoint via HuggingFace Hub from provided path ibm/biomed.sm.mv-te-84m
/Users/psuryan/Partha/work/code/envs/bmfm-sm-v5/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/psuryan/Partha/work/code/envs/bmfm-sm-v5/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
2024-10-26 10:27:54,534 - root - INFO - MacBookPro:8512582208:0:0 - Using coeff_mlp architecture for aggregator
2024-10-26 10:27:54,535 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:27:54,773 - root - INFO - MacBook

### Get Embeddings from a Pretrained Model

In [5]:
example_smiles = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"
example_emb = SmallMoleculeMultiViewModel.get_embeddings(
    smiles=example_smiles,
    model_path="ibm/biomed.sm.mv-te-84m",
    huggingface=True,
)
print(example_emb.shape)

2024-10-26 10:27:54,776 - root - INFO - MacBookPro:8512582208:0:0 - Loading checkpoint via HuggingFace Hub from provided path ibm/biomed.sm.mv-te-84m
2024-10-26 10:27:55,713 - root - INFO - MacBookPro:8512582208:0:0 - Using coeff_mlp architecture for aggregator
2024-10-26 10:27:55,713 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:27:55,813 - root - INFO - MacBookPro:8512582208:0:0 - in train False setting deterministic_eval = True


torch.Size([512])


### Load Finetuned Model from HuggingFace

In [6]:
dataset_registry = DatasetRegistry()
ds = dataset_registry.get_dataset_info('BACE')

In [7]:
finetuned_model_ds = SmallMoleculeMultiViewModel.from_finetuned(
    ds,
    model_path="ibm/biomed.sm.mv-te-84m-MoleculeNet-ligand_scaffold-BACE-101",
    inference_mode=True,
    huggingface=True
)

2024-10-26 10:27:56,058 - root - INFO - MacBookPro:8512582208:0:0 - Loading checkpoint via HuggingFace Hub from provided path ibm/biomed.sm.mv-te-84m-MoleculeNet-ligand_scaffold-BACE-101
2024-10-26 10:27:57,045 - root - INFO - MacBookPro:8512582208:0:0 - Using coeff_mlp architecture for aggregator
2024-10-26 10:27:57,046 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:27:57,303 - root - INFO - MacBookPro:8512582208:0:0 - in train False setting deterministic_eval = True


### Get Predictions from Finetuned Model

In [8]:
# Get predictions
prediction = SmallMoleculeMultiViewModel.get_predictions(
    example_smiles, ds, finetuned_model=finetuned_model_ds
)
print(prediction)

tensor(0, dtype=torch.int32)


## Using Local Data

You can also download the checkpoints and data to your system and use the API. Follow procedure in the README for setting up the data. Make sure that `BMFM_HOME` environment variable is pointing to the data_root.

In [9]:
print(os.environ['BMFM_HOME'])

/Users/psuryan/Partha/work/code/data/data_root_os_v1


### Loading Pretrained Model (Local)

Load our provided pretrained checkpoint, which uses a Attentional Late Fusion strategy (default). It loads the model in inference mode.

In [10]:
model_local = SmallMoleculeMultiViewModel.from_pretrained(LateFusionStrategy.ATTENTIONAL, inference_mode = True)

2024-10-26 10:27:58,357 - root - INFO - MacBookPro:8512582208:0:0 - Using coeff_mlp architecture for aggregator
2024-10-26 10:27:58,358 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:27:58,368 - root - INFO - MacBookPro:8512582208:0:0 - Loading checkpoint from default path /Users/psuryan/Partha/work/code/data/data_root_os_v1/bmfm_model_dir/pretrained/MULTIVIEW_MODEL/biomed-smmv-with-coeff-agg.pth
2024-10-26 10:27:58,466 - root - INFO - MacBookPro:8512582208:0:0 - Loading pretrain checkpoint for SmallMoleculeMultiView Model - <All keys matched successfully>
2024-10-26 10:27:58,472 - root - INFO - MacBookPro:8512582208:0:0 - in train False setting deterministic_eval = True


We can also load with a different fusion strategy - Note, aggregator will not be pretrained but BaseModel will be.

In [11]:
model_local = SmallMoleculeMultiViewModel.from_pretrained(LateFusionStrategy.MOE_WEIGHTED_CONCAT_PROJECTED, inference_mode = True)

2024-10-26 10:27:59,354 - root - INFO - MacBookPro:8512582208:0:0 - Using moe_weighted_concat architecture for aggregator
2024-10-26 10:27:59,354 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:27:59,358 - root - INFO - MacBookPro:8512582208:0:0 - Loading checkpoint from default path /Users/psuryan/Partha/work/code/data/data_root_os_v1/bmfm_model_dir/pretrained/MULTIVIEW_MODEL/biomed-smmv-with-coeff-agg.pth
2024-10-26 10:27:59,431 - root - INFO - MacBookPro:8512582208:0:0 - Loading pretrain checkpoint for SmallMoleculeMultiView Model - _IncompatibleKeys(missing_keys=['aggregator.weighted_concat_network.gating_network.fc.weight', 'aggregator.weighted_concat_network.gating_network.fc.bias'], unexpected_keys=['aggregator.w_before_mean.0.weight', 'aggregator.w_before_mean.0.bias', 'aggregator.w_before_mean.2.weight', 'aggregator.down_project.weight', 'aggregator.down_project.bias', 'aggregator.shared_task_head.0.weight', 'aggregator.shared_

We can also provide a `custom_model` path to load from a specific checkpoint

In [12]:
ckpt_path = f"{os.environ['BMFM_HOME']}/bmfm_model_dir/pretrained/MULTIVIEW_MODEL/biomed-smmv-base.pth"
model_local = SmallMoleculeMultiViewModel.from_pretrained(LateFusionStrategy.CONCAT, 
                                     model_path=ckpt_path,
                                     inference_mode = True)

2024-10-26 10:28:00,324 - root - INFO - MacBookPro:8512582208:0:0 - Using concat architecture for aggregator
2024-10-26 10:28:00,325 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:28:00,325 - root - INFO - MacBookPro:8512582208:0:0 - Loading checkpoint from provided path /Users/psuryan/Partha/work/code/data/data_root_os_v1/bmfm_model_dir/pretrained/MULTIVIEW_MODEL/biomed-smmv-base.pth
2024-10-26 10:28:00,421 - root - INFO - MacBookPro:8512582208:0:0 - Loading pretrain checkpoint for SmallMoleculeMultiView Model - <All keys matched successfully>
2024-10-26 10:28:00,425 - root - INFO - MacBookPro:8512582208:0:0 - in train False setting deterministic_eval = True


We can set `model_path` to `False` if to load a non-pretrained model.

In [13]:
model_local = SmallMoleculeMultiViewModel.from_pretrained(LateFusionStrategy.MOE_NOISED_WEIGHTED_CONCAT_BOTH_PROJECTED, 
                                     model_path=False,
                                     inference_mode = False)

2024-10-26 10:28:01,314 - root - INFO - MacBookPro:8512582208:0:0 - Using moe_noised_weighted_concat architecture for aggregator
2024-10-26 10:28:01,314 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:28:01,317 - root - INFO - MacBookPro:8512582208:0:0 - Not using checkpoint for model initialization


### Get Embeddings from a Pretrained Model (Local)

In [14]:
example_smiles = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"
example_emb = SmallMoleculeMultiViewModel.get_embeddings(example_smiles)
example_emb.shape

2024-10-26 10:28:02,199 - root - INFO - MacBookPro:8512582208:0:0 - Using coeff_mlp architecture for aggregator
2024-10-26 10:28:02,200 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:28:02,209 - root - INFO - MacBookPro:8512582208:0:0 - Loading checkpoint from default path /Users/psuryan/Partha/work/code/data/data_root_os_v1/bmfm_model_dir/pretrained/MULTIVIEW_MODEL/biomed-smmv-with-coeff-agg.pth
2024-10-26 10:28:02,278 - root - INFO - MacBookPro:8512582208:0:0 - Loading pretrain checkpoint for SmallMoleculeMultiView Model - <All keys matched successfully>
2024-10-26 10:28:02,283 - root - INFO - MacBookPro:8512582208:0:0 - in train False setting deterministic_eval = True


torch.Size([512])

### Load Finetuned Model (Local)

Choose a supported dataset for which finetuned checkpoint is available

In [15]:
#Example of a Classification Prediction
dataset_registry = DatasetRegistry()
bace_ds = dataset_registry.get_dataset_info('BACE')

In [16]:
finetuned_model_bace_local = SmallMoleculeMultiViewModel.from_finetuned(bace_ds, inference_mode = True)

2024-10-26 10:28:03,605 - root - INFO - MacBookPro:8512582208:0:0 - Using coeff_mlp architecture for aggregator
2024-10-26 10:28:03,605 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:28:03,925 - root - INFO - MacBookPro:8512582208:0:0 - Loading finetune checkpoint for SmallMoleculeMultiView Model - <All keys matched successfully>
2024-10-26 10:28:04,108 - root - INFO - MacBookPro:8512582208:0:0 - Loading finetune checkpoint for Prediction Head: /Users/psuryan/Partha/work/code/data/data_root_os_v1/bmfm_model_dir/finetuned/MULTIVIEW_MODEL/MoleculeNet/ligand_scaffold/BACE/best-101.ckpt
2024-10-26 10:28:04,136 - root - INFO - MacBookPro:8512582208:0:0 - in train False setting deterministic_eval = True


In [17]:
example_smiles = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"
bace_prediction = SmallMoleculeMultiViewModel.get_predictions(example_smiles, bace_ds, finetuned_model=finetuned_model_bace_local)
bace_prediction

tensor(0, dtype=torch.int32)

In [18]:
#Example of a Regression Prediction
esol_ds = dataset_registry.get_dataset_info('ESOL')
finetuned_model_esol_local = SmallMoleculeMultiViewModel.from_finetuned(esol_ds, inference_mode = True)
esol_prediction = SmallMoleculeMultiViewModel.get_predictions(example_smiles, esol_ds, finetuned_model=finetuned_model_esol_local)
esol_prediction

2024-10-26 10:28:05,556 - root - INFO - MacBookPro:8512582208:0:0 - Using coeff_mlp architecture for aggregator
2024-10-26 10:28:05,556 - root - INFO - MacBookPro:8512582208:0:0 - dim_list [512, 512, 768] for aggregator
2024-10-26 10:28:05,850 - root - INFO - MacBookPro:8512582208:0:0 - Loading finetune checkpoint for SmallMoleculeMultiView Model - <All keys matched successfully>
2024-10-26 10:28:06,021 - root - INFO - MacBookPro:8512582208:0:0 - Loading finetune checkpoint for Prediction Head: /Users/psuryan/Partha/work/code/data/data_root_os_v1/bmfm_model_dir/finetuned/MULTIVIEW_MODEL/MoleculeNet/ligand_scaffold/ESOL/best-101.ckpt
2024-10-26 10:28:06,039 - root - INFO - MacBookPro:8512582208:0:0 - in train False setting deterministic_eval = True


tensor(-3.5531)